In [ ]:
from azureml.core import Workspace, Experiment
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.runconfig import DataReferenceConfiguration, RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
import logging
# from sklearn.model_selection import train_test_split

# import pandas as pd
# import seaborn as sns
import os

PROJECT_FOLDER = './automl_scripts'
os.makedirs(PROJECT_FOLDER, exist_ok=True)

In [ ]:
EXPERIMENT_NAME = 'diabetes_regression_automl'

ws = Workspace.from_config()
experiment = Experiment(ws, EXPERIMENT_NAME)

cpu_cluster = ws.compute_targets['cpu-cluster']
ds = ws.datastores['diabetes']

In [ ]:
%%writefile $PROJECT_FOLDER/get_data.py
import os
import pandas as pd
from sklearn.model_selection import train_test_split

DATA_PATH = os.environ.get('DS_PATH')

def get_data():
    diabetes_df = pd.read_csv(os.path.join(DATA_PATH,"diabetes.csv"))

    y = diabetes_df.pop('target').values
    X = diabetes_df.values

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

    print (f"Data contains {len(X_train)} training samples and {len(X_test)} test samples")
    
    return {'X': X_train, 'X_valid': X_test, 'y': y_train, 'y_valid': y_test}

In [ ]:
dr = DataReferenceConfiguration(datastore_name=ds.name, 
                                mode='mount',
                                overwrite=False)

aml_rc = RunConfiguration(framework="python")

aml_rc.target = cpu_cluster
aml_rc.data_references = {ds.name: dr}
aml_rc.environment.environment_variables = {"DS_PATH": f"{ds.as_mount()}"}

aml_rc.environment.python.conda_dependencies = CondaDependencies.create(pip_packages=['azureml-sdk[automl]'], 
                                                                        conda_packages=['numpy', 'pandas'])


In [ ]:
automl_settings = {
    "iteration_timeout_minutes": 60,
    "iterations": 20,
    "preprocess": True,
    "max_cores_per_iteration": -1,
    "max_concurrent_iterations": 4,
    "verbosity": logging.INFO
}

aml_config = AutoMLConfig(task="regression",
                          path=PROJECT_FOLDER,
                          run_configuration=aml_rc,
                          data_script=PROJECT_FOLDER+'/get_data.py',
                          y_min=0,
                          **automl_settings
                         )

In [ ]:
run = experiment.submit(aml_config)
RunDetails(run).show()

#### Continue the runs if desired

In [ ]:
run.continue_experiment(iterations=5)
RunDetails(run).show()